## Statistical Learning and Deep Learning HW4

Load dataset.

In [1]:
# load packages
import pickle
from sklearn import preprocessing
%matplotlib inline

# load data
with open('msd_full.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

doscaling = 1
if doscaling == 1:
    xscaler = preprocessing.StandardScaler().fit(msd_data['X_train'])
    # standardize feature values
    X_train = xscaler.transform(msd_data['X_train'])
    X_test = xscaler.transform(msd_data['X_test'])
else:
    X_train = msd_data['X_train']
    X_test = msd_data['X_test']

Y_train = msd_data['Y_train']
Y_test = msd_data['Y_test'].astype('float32')
X_test = X_test.astype('float32')

y_mean = Y_train.mean()
Y_train_keep = Y_train.copy()
Y_test_keep = Y_test.copy()
Y_train = Y_train - y_mean
Y_test = Y_test - y_mean


# validation is the last 10% of training, subtraining is the first 90% of training
nvalid = int(X_train.shape[0] * 0.1)
nsubtrain = X_train.shape[0] - nvalid

X_subtrain = X_train[0:nsubtrain, :].astype('float32')
X_valid = X_train[nsubtrain:, :].astype('float32')
Y_subtrain = Y_train[0:nsubtrain].astype('float32')
Y_valid = Y_train[nsubtrain:].astype('float32')

Y_subtrain_keep = Y_train_keep[0:nsubtrain].astype('float32')
Y_valid_keep = Y_train_keep[nsubtrain:].astype('float32')

print("X_train shape = ", X_train.shape)
print("X_subtrain shape = ", X_subtrain.shape)
print("X_valid shape = ", X_valid.shape)
print()
print("Y_train shape = ", Y_train.shape)
print("Y_subtrain shape = ", Y_subtrain.shape)
print("Y_valid shape = ", Y_valid.shape)
print()
print("X_test shape = ", X_test.shape)
print("Y_test shape = ", Y_test.shape)

X_train shape =  (463715, 90)
X_subtrain shape =  (417344, 90)
X_valid shape =  (46371, 90)

Y_train shape =  (463715,)
Y_subtrain shape =  (417344,)
Y_valid shape =  (46371,)

X_test shape =  (51630, 90)
Y_test shape =  (51630,)


### Q1. Oridinary Least Square (OLS)

In [2]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

ols = sm.OLS(Y_train, X_train, hasconst=True)
ols_result = ols.fit()

In [3]:
print(f'The first 5 parameters: {ols_result.params[:5]}')

The first 5 parameters: [ 5.30975265 -2.88088114 -1.53234348  0.05737583 -0.33952889]


In [4]:
# predict
Y_predict = ols_result.predict(X_test)
print(f'The predicted Y is {Y_predict}')

The predicted Y is [-5.81070695  0.03250657  5.13960445 ... -1.39829429 -0.26047668
  0.05193056]


In [5]:
# RMSE
print(f'RMSE = {mean_squared_error(Y_test, Y_predict, squared=False)}')

RMSE = 9.510160684544402


### Q2. MLP with Four Hidden Layers

In [2]:
import numpy as np
import torch
from torch.utils import data
import os
from sklearn.metrics import mean_squared_error

print('torch version:', torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device}')
print(torch.cuda.current_device())

torch version: 1.7.1rc2
using cuda
0


In [3]:
# define dataset
class Dataset(data.Dataset):
    
  def __init__(self, Xnp, Ynp):
        self.labels = Ynp
        self.nobs = Xnp.shape[0]        
        self.Xnp = Xnp
        self.Ynp = Ynp
        
  def __len__(self):
        return self.nobs
    
  def __getitem__(self, index):     
        X = self.Xnp[index]
        y = self.Ynp[index]
        return X, y

In [4]:
# create dataloader
subtrain_set = Dataset(X_subtrain, Y_subtrain)    
valid_set = Dataset(X_valid, Y_valid)
test_set = Dataset(X_test, Y_test)
print('subtrain length', len(subtrain_set))
print('valid length', len(valid_set))
print('test length', len(test_set))

batch_size = 1000
subtrain_loader = data.DataLoader(subtrain_set, batch_size=batch_size)
valid_loader = data.DataLoader(valid_set, batch_size=batch_size)
test_loader = data.DataLoader(test_set, batch_size=batch_size)

subtrain length 417344
valid length 46371
test length 51630


In [5]:
# create MLP model
d_hidden = 45
d_input = subtrain_set.Xnp.shape[1]
d_output = 1

MLP = torch.nn.Sequential(
    torch.nn.Linear(d_input, d_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(d_hidden, d_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(d_hidden, d_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(d_hidden, d_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(d_hidden, d_output)
)
MLP.to(device)

Sequential(
  (0): Linear(in_features=90, out_features=45, bias=True)
  (1): ReLU()
  (2): Linear(in_features=45, out_features=45, bias=True)
  (3): ReLU()
  (4): Linear(in_features=45, out_features=45, bias=True)
  (5): ReLU()
  (6): Linear(in_features=45, out_features=45, bias=True)
  (7): ReLU()
  (8): Linear(in_features=45, out_features=1, bias=True)
)

In [6]:
# optimizer
lr = 0.00001
momentum = 0
weight_decay = 0
optimizer = torch.optim.SGD(MLP.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

# loss
loss_func = torch.nn.MSELoss(reduction='sum')

In [7]:
# train

def RMSE(data_loader):
    n_obs = 0
    sse = 0
    for _batch, (_inputs, _targets) in enumerate(subtrain_loader):
        _targets = _targets.reshape((-1, 1))
        _inputs, _targets = _inputs.to(device), _targets.to(device)
        _outputs = MLP(_inputs)
        n_obs += _targets.shape[0]
        sse += (_targets - _outputs).pow(2).sum(0)
    return np.sqrt(sse.cpu().numpy()[0] / n_obs)

    
def train(MLP, max_epoch, max_step, valid_interval, weight_path, train_rmse=False):
    
    step_count = 0
    best_step_count = 0
    best_valid_rmse = np.inf
    all_train_rmse = []
    all_valid_rmse = []
    all_step = []

    for epoch in range(1, max_epoch+1):
        for batch, (inputs, targets) in enumerate(subtrain_loader):

            targets = targets.reshape((-1, 1))
            inputs, targets = inputs.to(device), targets.to(device)
            MLP.to(device)
            MLP.train()
            optimizer.zero_grad()
            outputs = MLP(inputs)
            loss = loss_func(outputs, targets)
            loss.backward()
            optimizer.step()
            step_count += 1

            # check train/validation RMSE
            if (step_count % valid_interval == 0):

                with torch.no_grad():
                    # subtrain, validation RMSE
                    if train_rmse:
                        train_rmse = RMSE(subtrain_loader)
                        all_train_rmse.append(train_rmse)
                    valid_rmse = RMSE(valid_loader)
                    all_valid_rmse.append(valid_rmse)
                    all_step.append(step_count)

                    # update weight
                    if valid_rmse < best_valid_rmse:
                        best_step_count = step_count
                        best_valid_rmse = valid_rmse
                        torch.save(MLP.state_dict(), weight_path)
                        
                    # early stopping
                    elif (step_count - best_step_count >= max_step):
                        print(f'early stopping, step {step_count}, validation RMSE = {valid_rmse}')
                        return all_step, all_train_rmse, all_valid_rmse

        print(f'Epoch {epoch}, Step {step_count}: Loss = {loss.item()}, best_valid_rmse = {best_valid_rmse}')
    return all_step, all_train_rmse, all_valid_rmse

In [ ]:
all_step, all_train_rmse, all_valid_rmse = train(MLP=MLP, max_epoch=100, max_step=5000, valid_interval=100, weight_path='./MLP45_weight', train_rmse=True)        


Epoch 1, Step 418: Loss = 35491.0, best_valid_rmse = 8.9548867520085
Epoch 2, Step 836: Loss = 34541.1796875, best_valid_rmse = 8.713036698589338
Epoch 3, Step 1254: Loss = 34125.15625, best_valid_rmse = 8.671243370223578


In [ ]:
import matplotlib.pyplot as plt

# plot subtrain / validation RMSE
plt.plot(all_step, all_train_rmse, c='b', label='Training RMSE')
plt.plot(all_step, all_valid_rmse, c='r', label='Validation RMSE')
plt.legend()
plt.xlabel('Batch')
plt.ylabel('RMSE')
plt.title('Training & Validation RMSE')

In [ ]:
# test RMSE
with torch.no_grad():
    test_rmse = RMSE(test_loader)
print(f'Test RMSE = {test_rmse}')

### Q3. H = 90, 180